In [1]:
!git clone https://github.com/usefulsensors/moonshine.git

fatal: destination path 'moonshine' already exists and is not an empty directory.


In [2]:
cd moonshine

c:\Users\karth\OneDrive\Desktop\BlueKyte Work\VTT\moonshine


In [3]:
!python3 -m venv env

Error: [WinError 2] The system cannot find the file specified


In [4]:
!source env/bin/activate

In [4]:
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [6]:
import moonshine

In [7]:
model = moonshine.load_model("moonshine/base")

In [9]:
result_1 = moonshine.transcribe(r"C:\Users\karth\OneDrive\Desktop\BlueKyte Work\VTT\KB.wav", model=model)

c:\Users\karth\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (1, 8, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


In [10]:
print(result_1)

['Can you go to case research and search for case one on the back of the substrate of Kerala case and give you the summary edition of the case?']


In [16]:
pip install aioboto3 boto3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 109.3 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.35.43
    Uninstalling botocore-1.35.43:
      Successfully uninstalled botocore-1.35.43
  Attempting uninstall: boto3
    Found existing installation: boto3 1.35.43
    Uninstalling boto3-1.35.43:
      Successfully uninstalled boto3-1.35.43
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.35.9 requires botocore==1.35.43, but you have botocore 1.35.36 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [17]:
import nest_asyncio
import asyncio
import moonshine
import json
import aioboto3
from botocore.config import Config

In [18]:
nest_asyncio.apply()

In [19]:
# Load Moonshine model
model = moonshine.load_model("moonshine/base")

In [ ]:
# Define the Bedrock function for text correction
async def query_bedrock(context_prompt, stream_flag, model):
    session = aioboto3.Session()
    async with session.client(
        service_name="bedrock-runtime",
        region_name="us-east-1",
        aws_access_key_id="",
        aws_secret_access_key="",
        config=Config(
            connect_timeout=300, read_timeout=300, retries={"max_attempts": 10}
        ),
    ) as bedrock:

        body = json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "messages": [
                {
                    "role": "user",
                    "content": [{"type": "text", "text": context_prompt}],
                }
            ],
            "max_tokens": 2000,
            "temperature": 0.5,
            "top_p": 0.9,
        })

        accept = "application/json"
        contentType = "application/json"

        if stream_flag:
            response = await bedrock.invoke_model_with_response_stream(
                body=body, modelId=model, accept=accept, contentType=contentType
            )
            async for event in response["body"]:
                chunk = event.get("chunk")
                if chunk:
                    obj = json.loads(chunk.get("bytes").decode())
                    yield f"data: {json.dumps(obj)}\n\n"
        else:
            response = await bedrock.invoke_model(
                body=body, modelId=model, accept=accept, contentType=contentType
            )
            response_body = json.loads(await response.get("body").read())
            text_content = response_body.get("content", [])[0].get("text", "")
            yield text_content

In [ ]:
# Main function to transcribe audio with Moonshine and correct text using Bedrock
async def main(audio_file_path):
    # Step 1: Transcribe audio using Moonshine
    print("\nTranscribing audio... Please wait...")
    transcription = moonshine.transcribe(audio_file_path, model=model)
    print("\nInitial Transcription:")
    print("-" * 50)
    print(transcription)
    print("-" * 50)

    # Step 2: Apply Bedrock correction
    print("\nSending transcription for correction... Please wait...")
    
    # Define the correction prompt
    correction_prompt = f"""Please review and correct only the Indian proper nouns, names, and legal case names in the following text. 
    Ensure that any Indian legal case names, personal names, or geographical names are spelled correctly and formatted according to standard Indian legal conventions.

    Original text: {transcription}

    Strict Correction Rules:
    - Only correct Indian case names, personal names, or geographical names, keeping all other text exactly the same.
    - Maintain accurate spelling and capitalization for names related to Indian law, including proper noun formats.
    - Do not rephrase, reword, or modify non-Indian names, any common words, or any parts of the text other than the specified proper nouns.
    - Ensure case titles adhere to Indian legal citation formats, such as "XYZ vs. ABC" or "In re: XYZ."
    - If unsure of a correction, leave the text exactly as it is.

    Output should contain only the strictly corrected text, with no added notes or explanations.
    """


    # Query Bedrock
    model_id = "anthropic.claude-3-haiku-20240307-v1:0"  # Replace with our in-use model ID
    async for corrected_text in query_bedrock(correction_prompt, stream_flag=False, model=model_id):
        print("\nCorrected Transcription:")
        print("-" * 50)
        print(corrected_text)
        print("-" * 50)

In [ ]:
# audio_file_path = "/home/ec2-user/SageMaker/KB.wav"
# audio_file_path = "/home/ec2-user/SageMaker/updated_scindia.wav"
audio_file_path = "/home/ec2-user/SageMaker/updated_scindia.wav"
scindia_clear.wav

await main(audio_file_path)


Transcribing audio... Please wait...

Initial Transcription:
--------------------------------------------------
['He head over to Casey Search and search for Martha Jivaji Rouse India vs. Union of India, Nineteen Seventy.']
--------------------------------------------------

Sending transcription for correction... Please wait...

Corrected Transcription:
--------------------------------------------------
['He head over to Casey Search and search for Mahatma Jivaji Rau v. Union of India, 1970.']
--------------------------------------------------
